# Chapter 7 Dictionary Tricks

# 7.1 Dictionary Default Values
假設我們想建立客製化的 greeting function
+ 不同 user 而有不同打招呼方式
+ 碰上 KeyError 也能正常運作
  
以下的寫法看似很好，但有些缺點
+ 查找了2次字典
+ 冗長(verbose)，Hi 字串重複了2次
+ It’s not Pythonic，跟 python 文檔的 “easier to ask for forgiveness than
permission“(EAFP) coding style 違背

> This common Python coding style assumes the existence of valid keys or attributes and catches exceptions
if the assumption proves false

In [ ]:
name_for_userid = {
  382: 'Alice',
  950: 'Bob',
  590: 'Dilbert',
}

def greeting(userid):
  if userid in name_for_userid:
    return 'Hi %s!' % name_for_userid[userid]
  else:
    return 'Hi there!'

In [ ]:
print(greeting(382))
greeting(999)

Hi Alice!


'Hi there!'

符合 EAFP 原則的寫法，運用 try except 來處理 KeyError

In [ ]:
def greeting(userid):
  try:
    return 'Hi %s!' % name_for_userid[userid]
  except KeyError:
    return 'Hi there'

更好的寫法:  
利用 get 來在 KeyError 時回傳預設值

In [ ]:
def greeting(userid):
  return 'Hi %s!' % name_for_userid.get(
  userid, 'there')

**Key Takeaways**
+ Avoid explicit key in dict checks when testing for membership.
+ EAFP-style exception handling or using the built-in get()
method is preferable.
+ In some cases, the collections.defaultdict class from the
standard library can also be helpful.

# 7.2 Sorting Dictionaries for Fun and Profit
你可以對 python dictionary 進行 iterate，但順序是無法確定的  
  
我們可以對字典內的 items 先進行默認排序 (比較字典的 key)

In [1]:
xs = {'a': 4, 'c': 2, 'b': 3, 'd': 1}
sorted(xs.items())

[('a', 4), ('b', 3), ('c', 2), ('d', 1)]

但如果想要自訂排序基準可使用 key function 來指定，範例指定 value 為基準並進行翻轉

In [2]:
sorted(xs.items(), key=lambda x: x[1], reverse=True)

[('d', 1), ('c', 2), ('b', 3), ('a', 4)]

也能用已經實作的 operator.itemgetter 和 operator.attrgetter 來替代  
但作者建議使用 lambda 方式因為可讀性更高

In [3]:
import operator
sorted(xs.items(), key=operator.itemgetter(1))

[('d', 1), ('c', 2), ('b', 3), ('a', 4)]

**Key Takeaways**
+ When creating sorted “views” of dictionaries and other collections, you can influence the sort order with a key func.
+ Key funcs are an important concept in Python. The most frequently used ones were even added to the operator module in
the standard library.
+ Functions are first-class citizens in Python. This is a powerful
feature you’ll find used everywhere in the language.

# 7.3 Emulating Switch/Case Statements With Dicts
python 不支援 switch case，導致有時必須寫出一堆的 if 判斷句

```python
if cond == 'cond_a':
  handle_a()
elif cond == 'cond_b':
  handle_b()
else:
  handle_default()
```

但我們能用 first-class functions 來改善這些 if 描述  
first-class functions 意思是 function 能跟一般變數一樣，被作為參數傳遞或是當成回傳值

```python
func_dict = {
  'cond_a': handle_a,
  'cond_b': handle_b
}

cond = 'cond_a'
func_dict[cond]()

# handle default 
func_dict.get(cond, handle_default)()
```

以下的兩種寫法都能把原本冗長的 if 敘述給抽離，他們具有一樣的功能  
  
但還有進度的空間
+ 例如 dispatch_dict 每次呼叫都會宣告一個暫時的字典和好幾個 lambda，更好的方式為把字典放在 function 外面  
+ 使用 python 內建 operator 像是 operator.add, operator.mul 來取代 lambda function


In [8]:
def dispatch_if(operator, x, y):
  if operator == 'add':
    return x + y
  elif operator == 'sub':
    return x - y
  elif operator == 'mul':
    return x * y
  elif operator == 'div':
    return x / y

In [33]:
def dispatch_dict(op, x, y):
  return {
    'add': lambda: x + y,
    'sub': lambda: x - y,
    'mul': lambda: x * y,
    'div': lambda: x / y,
  }.get(op, lambda: None)()

**Key Takeaways**
+ Python doesn’t have a switch/case statement. But in some cases
you can avoid long if-chains with a dictionary-based dispatch
table.
+ Once again Python’s first-class functions prove to be a powerful
tool. But with great power comes great responsibility.

# 7.4 The Craziest Dict Expression in the West
讓我們來看一下以下的字典宣告會產生什麼結果

In [35]:
xs = {True: 'yes', 1: 'no', 1.0: 'maybe'}
xs

{True: 'maybe'}

竟然只剩下一個item，這是因為 python interpreter 將其編譯為以下程式碼
```python
xs = dict()
xs[True] = 'yes'
xs[1] = 'no'
xs[1.0] = 'maybe'
```
而三個 key 又是等效的，只會不斷更新 key 對應的數值  
  
根據 [python document](https://docs.python.org/3/reference/datamodel.html#the-standard-type-hierarchy)
> “The Boolean type is a subtype of the integer type, and
Boolean values behave like the values 0 and 1, respectively, in almost all contexts, the exception being that
when converted to a string, the strings ‘False’ or ‘True’
are returned, respectively

```python
True == 1 == 1.0
>>> True
```
意思是你也能用 True / False 代表 0/1
```python
['no', 'yes'][True]
>>> 'yes'
```




python 字典的 key 有兩種比較方式，`__eq__` 和 `__hash__`，只有在兩者**都相同**的情況下才會覆蓋數值

只有 `__eq__` 相同，字典視為不同 key 值

In [52]:
class AlwaysEquals:
  def __eq__(self, other):
    return True
  def __hash__(self):
    return id(self)

In [53]:
print(AlwaysEquals() == AlwaysEquals())
print(AlwaysEquals() == 42)

True
True


In [54]:
objects = [AlwaysEquals(),
      AlwaysEquals(),
      AlwaysEquals()]

[hash(obj) for obj in objects]

[140228725248592, 140228725248848, 140228725249040]

In [55]:
{AlwaysEquals(): 'yes', AlwaysEquals(): 'no'}

{<__main__.AlwaysEquals at 0x7f898b599090>: 'no',
 <__main__.AlwaysEquals at 0x7f898b599890>: 'yes'}

只有 hash 相同，字典視為不同 key 值

In [57]:
class SameHash:
  def __hash__(self):
    return 1

In [58]:
a = SameHash()
b = SameHash()
print(a == b)

False


In [59]:
{a: 'a', b: 'b'}

{<__main__.SameHash at 0x7f898b593210>: 'a',
 <__main__.SameHash at 0x7f89989a75d0>: 'b'}

**Key Takeaways**
+ Dictionaries treat keys as identical if their \_\_eq\_\_ comparison
result says they’re equal and their hash values are the same.
+ Unexpected dictionary key collisions can and will lead to surprising results.

# 7.5 So Many Ways to Merge Dictionaries
這章節將講述整合字典的方法  
  
最後加入的字典級別較高，能覆蓋舊的數值

In [64]:
xs = {'a': 1, 'b': 2}
ys = {'b': 3, 'c': 4}

In [65]:
zs = {}
zs.update(xs)
zs.update(ys)
zs

{'a': 1, 'b': 3, 'c': 4}

Python 2 和 Python 3 有內建的整合字典方式，但是只限於整合兩個字典

In [67]:
zs = dict(xs, **ys)
zs

{'a': 1, 'b': 3, 'c': 4}

到了 python 3.5，\*\*符號更實用了

In [68]:
zs = {**xs, **ys}
zs

{'a': 1, 'b': 3, 'c': 4}

**Key Takeaways**
+ In Python 3.5 and above you can use the \*\*-operator to merge
multiple dictionary objects into one with a single expression,
overwriting existing keys left-to-right.
+ To stay compatible with older versions of Python, you might
want to use the built-in dictionary update() method instead.

# 7.6 Dictionary Pretty-Printing
預設的字典-字串轉換可讀性很差，順序不一且沒有換行，這導致開發上的困難

In [71]:
mapping = {'a': 23, 'b': 42, 'c': 0xc0ffee}
str(mapping)

"{'a': 23, 'b': 42, 'c': 12648430}"

json 可以解決此問題，但並不完美
+ key 數值只能是 primitive types
+ value 不能是 complex data types
+ 印出來的 Unicode text 跟原本格式不同

In [74]:
import json
print(json.dumps(mapping, indent=4, sort_keys=True))

{
    "a": 23,
    "b": 42,
    "c": 12648430
}


In [75]:
mapping = {0:1, 1:2}
json.dumps(mapping)

'{"0": 1, "1": 2}'

In [77]:
# TypeError: keys must be str, int, float, bool or None, not builtin_function_or_method
# json.dumps({all: 'yup'})

In [83]:
# TypeError: Object of type set is not JSON serializable
# json.dumps({0: set([1,2,3])})

In [79]:
json.dumps({0: '你好嗎'})

'{"0": "\\u4f60\\u597d\\u55ce"}'

pprint 能解決以上問題，但在可讀性上沒這麼高

In [81]:
import pprint

mapping = {'a':23,
      'b':20,
      'c':set([1,2,3])
      }

pprint.pprint(mapping)

{'a': 23, 'b': 20, 'c': {1, 2, 3}}


**Key Takeaways**
+ The default to-string conversion for dictionary objects in
Python can be difficult to read.
+ The pprint and json module are “higher-fidelity” options built
into the Python standard library.
+ Be careful with using json.dumps() and non-primitive keys
and values as this will trigger a TypeError.